Projeto Loteca model XGBoost

#Bibliotecas

In [2]:
import pandas as pd
import numpy as np
from textwrap import shorten
from collections import deque
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import os
import glob
import re

Dados 2016

In [ ]:
# --- ETAPA 1: CARREGAMENTO DOS ARQUIVOS ---
caminho_serie_a = 'dados/brasileiraoA/brasileiraoA2016.csv'
caminho_serie_b = 'dados/brasileiraoB/brasileiraoB2016.csv'
caminho_times = 'dados/times/times2006.csv'
caminho_copa_brasil = 'dados/copadobrasil/copadobrasil2016.csv'
caminho_libertadores = 'dados/libertadores/libertadores2016.csv'
caminho_sudamericana = 'dados/sudamericana/sudamericana2016.csv'
try:
    df_serie_a = pd.read_csv(caminho_serie_a)
    df_serie_b = pd.read_csv(caminho_serie_b)
    df_times = pd.read_csv(caminho_times)
    df_sudamericana = pd.read_csv(caminho_sudamericana)
    # Carrega os arquivos de copa e mostra as colunas para debug
    df_copa_brasil = pd.read_csv(caminho_copa_brasil)
    df_libertadores = pd.read_csv(caminho_libertadores)
    
    print("✅ Arquivos CSV carregados com sucesso!")
    print(f"Número de registros Copa do Brasil: {len(df_copa_brasil)}")
    print(f"Número de registros Libertadores: {len(df_libertadores)}")
    print(f"Número de registros Sudamericana: {len(df_sudamericana)}")
    print(f"Número de registros Série A: {len(df_serie_a)}")
    print(f"Número de registros Série B: {len(df_serie_b)}")
    print(f"Número de registros Times: {len(df_times)}")
    
except FileNotFoundError as e:
    print(f"❌ Erro: Arquivo não encontrado! Verifique o caminho: {e.filename}")
    exit()

# --- ETAPA 2: PROCESSAMENTO DOS DADOS DE COPA ---
# --- FUNÇÃO PARA PROCESSAR JOGOS DE COPA ---
def processar_jogos_copa(df_copa, competicao):
    """Processa os jogos de copa e retorna um dicionário com informações por time"""
    jogos_por_time = {}
    
    # Verifica os nomes das colunas e ajusta conforme necessário
    coluna_mandante = None
    coluna_visitante = None
    coluna_data = None
    coluna_fase = None
    
    # Mapeia possíveis nomes de colunas
    possiveis_colunas = {
        'mandante': ['Time Mandante', 'Mandante', 'Time da Casa', 'Casa'],
        'visitante': ['Time Visitante', 'Visitante', 'Time de Fora', 'Fora'],
        'data': ['Data', 'Date', 'Dia'],
        'fase': ['Fase', 'Phase', 'Stage', 'Rodada']
    }
    
    for col in df_copa.columns:
        col_lower = col.lower()
        if any(x.lower() in col_lower for x in possiveis_colunas['mandante']):
            coluna_mandante = col
        elif any(x.lower() in col_lower for x in possiveis_colunas['visitante']):
            coluna_visitante = col
        elif any(x.lower() in col_lower for x in possiveis_colunas['data']):
            coluna_data = col
        elif any(x.lower() in col_lower for x in possiveis_colunas['fase']):
            coluna_fase = col
    
    if not all([coluna_mandante, coluna_visitante, coluna_data]):
        print(f"❌ Colunas não encontradas em {competicao}")
        return jogos_por_time
    
    for _, jogo in df_copa.iterrows():
        try:
            # Converte a data para formato datetime (tenta diferentes formatos)
            data_str = str(jogo[coluna_data])
            data_jogo = None
            
            # Tenta diferentes formatos de data
            formatos_data = ['%d/%m/%y', '%d/%m/%Y', '%Y-%m-%d', '%m/%d/%Y']
            for formato in formatos_data:
                try:
                    data_jogo = datetime.strptime(data_str, formato)
                    break
                except ValueError:
                    continue
            
            if data_jogo is None:
                continue
            
            # Adiciona informações para o time mandante
            mandante = str(jogo[coluna_mandante]).strip()
            if mandante and mandante != 'nan':
                if mandante not in jogos_por_time:
                    jogos_por_time[mandante] = []
                
                fase = str(jogo[coluna_fase]).strip() if coluna_fase else 'F'
                jogos_por_time[mandante].append({
                    'data': data_jogo,
                    'competicao': competicao,
                    'fase': fase,
                    'adversario': str(jogo[coluna_visitante]).strip(),
                    'local': 'casa'
                })
            
            # Adiciona informações para o time visitante
            visitante = str(jogo[coluna_visitante]).strip()
            if visitante and visitante != 'nan':
                if visitante not in jogos_por_time:
                    jogos_por_time[visitante] = []
                
                fase = str(jogo[coluna_fase]).strip() if coluna_fase else 'F'
                jogos_por_time[visitante].append({
                    'data': data_jogo,
                    'competicao': competicao,
                    'fase': fase,
                    'adversario': str(jogo[coluna_mandante]).strip(),
                    'local': 'fora'
                })
            
        except (ValueError, KeyError) as e:
            continue
    
    return jogos_por_time

# Processa os jogos de copa
jogos_copa_brasil = processar_jogos_copa(df_copa_brasil, 'Copa do Brasil')
jogos_libertadores = processar_jogos_copa(df_libertadores, 'Libertadores')
jogos_sudamericana = processar_jogos_copa(df_sudamericana, 'Sudamericana')
    
# Combina os dois dicionários
todos_jogos_copa = {}
for time in set(list(jogos_copa_brasil.keys()) + list(jogos_libertadores.keys()) + list(jogos_sudamericana.keys())):
    todos_jogos_copa[time] = (jogos_copa_brasil.get(time, []) + 
                             jogos_libertadores.get(time, []) + 
                             jogos_sudamericana.get(time, []))

# Ordena os jogos por data para cada time
for time in todos_jogos_copa:
    todos_jogos_copa[time].sort(key=lambda x: x['data'])

def normalizar_times(nome):
    nome = str(nome).strip()
    substituicoes = {
        'Athletico-PR': 'Atlético-PR', 
        'Athletico Paranaense': 'Atlético-PR',
        'Ceará SC': 'Ceará', 
        'Sport Recife': 'Sport',
        'Vasco da Gama': 'Vasco', 
        'América Mineiro': 'América-MG',
        'Atlético Mineiro': 'Atlético-MG', 
        'Atlético Goianiense': 'Atlético-GO',
        'Red Bull Bragantino': 'Bragantino',
        'Grêmio Novorizontino': 'Novorizontino'
    }
    return substituicoes.get(nome, nome)

# Mostra estatísticas dos jogos de copa processados
print(f"\n📊 Estatísticas dos jogos de copa processados:")
print(f"Times na Copa do Brasil: {len(jogos_copa_brasil)}")
print(f"Times na Libertadores: {len(jogos_libertadores)}")
print(f"Times na Sudamericana: {len(jogos_sudamericana)}")
print(f"Times totais com jogos de copa: {len(todos_jogos_copa)}")

Carregar Simulação 2016


In [ ]:
# 1. Caminho para a pasta onde estão os arquivos
caminho_pasta = 'simulacao/2016/'

# 2. Busca todos os arquivos que começam com 'rodada' e terminam com '.csv'
# O '*' funciona como um coringa (wildcard)
arquivos_rodadas = glob.glob(os.path.join(caminho_pasta, 'rodada*.csv'))

# Ordenar os arquivos numericamente para garantir que a sequência esteja correta
# (Opcional, mas boa prática para conferência)
arquivos_rodadas.sort()

print(f"📂 Encontrados {len(arquivos_rodadas)} arquivos de rodada.")

# 3. Lista para armazenar os DataFrames de cada rodada
lista_dfs = []

for arquivo in arquivos_rodadas:
    try:
        df_temp = pd.read_csv(arquivo)
        # É sempre bom garantir que a coluna 'Rodada' exista e esteja correta
        # Se não tiver no CSV, você pode extrair do nome do arquivo
        lista_dfs.append(df_temp)
    except Exception as e:
        print(f"⚠️ Erro ao ler {arquivo}: {e}")

# 4. Concatenar (empilhar) todos os DataFrames em um só
df_serie_a_completo = pd.concat(lista_dfs, ignore_index=True)

# 5. Ordenar por rodada para o seu código de análise de features funcionar perfeitamente
df_serie_a_completo = df_serie_a_completo.sort_values(by='Rodada').reset_index(drop=True)

print(f"✅ Consolidação concluída! Total de jogos carregados: {len(df_serie_a_completo)}")
print(df_serie_a_completo)

Preparar Features

In [ ]:
from collections import deque
import pandas as pd
import numpy as np

def gerar_features_completas(df_jogos, df_times):
    print("\n--- Integrando Sinais: Momentum, Força e Desespero ---")
    COLUNA_MANDANTE = 'Time da Casa'
    COLUNA_VISITANTE = 'Time Visitante'
    df_jogos = df_jogos.sort_values(by='Rodada').reset_index(drop=True)
    
    def extrair_gols(placar):
        try:
            gols = str(placar).replace('–', '-').split('-')
            return int(gols[0]), int(gols[1])
        except: return 0, 0
            
    df_jogos['Gols_Mandante'] = df_jogos['Placar'].apply(lambda p: extrair_gols(p)[0])
    df_jogos['Gols_Visitante'] = df_jogos['Placar'].apply(lambda p: extrair_gols(p)[1])
    
    media_gols_liga = df_jogos['Gols_Mandante'].mean() if not df_jogos.empty else 1.2
    
    try:
        df_jogos['Data_Datetime'] = pd.to_datetime(df_jogos['Data'], format='%d/%m/%y', errors='coerce')
    except:
        df_jogos['Data_Datetime'] = pd.to_datetime(df_jogos['Data'], errors='coerce')
    
    mapa_time_regiao = df_times.set_index('time')['região'].to_dict()
    unique_teams = pd.concat([df_jogos[COLUNA_MANDANTE], df_jogos[COLUNA_VISITANTE]]).unique()
    
    stats_times = {time: {
        'pontos': 0, 'jogos': 0, 'vitorias': 0, 'sg_casa': 0, 'sg_fora': 0,
        'gols_marcados_casa': 0, 'gols_sofridos_casa': 0, 'jogos_casa': 0,
        'gols_marcados_fora': 0, 'gols_sofridos_fora': 0, 'jogos_fora': 0,
        'ultimos_5_saldos_casa': deque(maxlen=5),
        'ultimos_5_saldos_fora': deque(maxlen=5),
        'ultimos_5_resultados': deque(maxlen=5)
    } for time in unique_teams}
    

    listas_features = {
        'Eh_Serie_B': [],
        'Posicao_Mandante': [], 'Posicao_Visitante': [],
        'Media_GM_Casa': [], 'Media_GS_Casa': [],
        'Media_GM_Fora': [], 'Media_GS_Fora': [],
        'Saldo_Gols_Casa_Mandante': [], 'Saldo_Gols_Fora_Visitante': [],
        'Saldo_Ultimos_5_Casa_Mandante': [], 'Saldo_Ultimos_5_Fora_Visitante': [],
        'Sequencia_5_Mandante': [], 'Sequencia_5_Visitante': [],
        'Proxima_Copa_Mandante': [], 'Proxima_Copa_Visitante': [],
        'Forca_Atk_M': [], 'Forca_Def_V': [], 'Sinal_Dominio': [],
        'Momentum_M': [], 'Momentum_V': [],
        'Desespero_Mandante': [], 'Desespero_Visitante': [], 'Delta_Desespero': [],
        'Soberba_Mandante': [], 'Soberba_Visitante': [], 'Delta_Soberba': [],
    }
    

    def calcular_momentum_ia(deque_resultados):
        if not deque_resultados: return 0
        pontos_map = {'V': 3, 'E': 1, 'D': 0}
        vals = [pontos_map.get(res, 0) for res in deque_resultados]
        pesos = range(1, len(vals) + 1)
        return sum(v * p for v, p in zip(vals, pesos)) / sum(pesos)
    
    for rodada in range(1, 39):
        df_classificacao = pd.DataFrame.from_dict(stats_times, orient='index')
        df_classificacao['sg_total'] = df_classificacao['sg_casa'] + df_classificacao['sg_fora']
        df_classificacao = df_classificacao.sort_values(by=['pontos', 'vitorias', 'sg_total'], ascending=False)
        df_classificacao['posicao'] = range(1, len(df_classificacao) + 1)
        mapa_posicao = df_classificacao['posicao'].to_dict()
        
        
        jogos_da_rodada = df_jogos[df_jogos['Rodada'] == rodada]
        
        for index, jogo in jogos_da_rodada.iterrows():
            mandante, visitante = jogo[COLUNA_MANDANTE], jogo[COLUNA_VISITANTE]
            data_jogo = jogo['Data_Datetime']
            if pd.isna(data_jogo): continue
            
            stats_m, stats_v = stats_times[mandante], stats_times[visitante]
            
            serie_b_sinal = jogo['Eh_Serie_B'] if 'Eh_Serie_B' in jogo else 0
            listas_features['Eh_Serie_B'].append(serie_b_sinal)
            
            # --- CÁLCULO DAS FEATURES (ANTES DO JOGO) ---
            pos_m = mapa_posicao.get(mandante, 21)
            pos_v = mapa_posicao.get(visitante, 21)
            listas_features['Posicao_Mandante'].append(pos_m)
            listas_features['Posicao_Visitante'].append(pos_v)
            
            # --- FEATURE: DESESPERO ---
            def calcular_desespero(pos, rd):
                if rd < 10: return 0
                urgencia = 0
                if pos <= 3: urgencia = 1.0       # Título
                elif pos <= 8: urgencia = 0.7     # Libertadores
                elif pos >= 17: urgencia = 1.2    # REBAIXAMENTO
                elif pos >= 14: urgencia = 0.5    # Alerta
                return urgencia * (rd / 38)

            d_m = calcular_desespero(pos_m, rodada)
            d_v = calcular_desespero(pos_v, rodada)
            listas_features['Desespero_Mandante'].append(d_m)
            listas_features['Desespero_Visitante'].append(d_v)
            listas_features['Delta_Desespero'].append(d_m - d_v)
            
            
               
            # Médias e Forças
            mgm_c = stats_m['gols_marcados_casa'] / stats_m['jogos_casa'] if stats_m['jogos_casa'] > 0 else 0
            mgs_f = stats_v['gols_sofridos_fora'] / stats_v['jogos_fora'] if stats_v['jogos_fora'] > 0 else 0
            
            listas_features['Media_GM_Casa'].append(mgm_c)
            listas_features['Media_GS_Casa'].append(stats_m['gols_sofridos_casa'] / stats_m['jogos_casa'] if stats_m['jogos_casa'] > 0 else 0)
            listas_features['Media_GM_Fora'].append(stats_v['gols_marcados_fora'] / stats_v['jogos_fora'] if stats_v['jogos_fora'] > 0 else 0)
            listas_features['Media_GS_Fora'].append(mgs_f)
            
            listas_features['Saldo_Gols_Casa_Mandante'].append(stats_m['sg_casa'])
            listas_features['Saldo_Gols_Fora_Visitante'].append(stats_v['sg_fora'])
            listas_features['Saldo_Ultimos_5_Casa_Mandante'].append(sum(stats_m['ultimos_5_saldos_casa']))
            listas_features['Saldo_Ultimos_5_Fora_Visitante'].append(sum(stats_v['ultimos_5_saldos_fora']))
            
            listas_features['Sequencia_5_Mandante'].append(''.join(stats_m['ultimos_5_resultados']) or '-')
            listas_features['Sequencia_5_Visitante'].append(''.join(stats_v['ultimos_5_resultados']) or '-')

            # Sinais de Força
            f_atk = mgm_c / media_gols_liga
            f_def = mgs_f / media_gols_liga
            listas_features['Forca_Atk_M'].append(f_atk)
            listas_features['Forca_Def_V'].append(f_def)
            listas_features['Sinal_Dominio'].append(f_atk * f_def)
            listas_features['Momentum_M'].append(calcular_momentum_ia(stats_m['ultimos_5_resultados']))
            listas_features['Momentum_V'].append(calcular_momentum_ia(stats_v['ultimos_5_resultados']))

            # Lógica de Copa
            def get_proxima_copa(time, data_atual):
                if 'todos_jogos_copa' in globals() and time in todos_jogos_copa:
                    proximos = [j for j in todos_jogos_copa[time] if j['data'] > data_atual]
                    if proximos:
                        prox = min(proximos, key=lambda x: x['data'])
                        if (prox['data'] - data_atual).days <= 7:
                            return f"{prox['competicao'][0]}{prox['fase'][0]}"
                return '-'
            
            listas_features['Proxima_Copa_Mandante'].append(get_proxima_copa(mandante, data_jogo))
            listas_features['Proxima_Copa_Visitante'].append(get_proxima_copa(visitante, data_jogo))
            
            def calcular_soberba(tem_copa, pos_time, pos_adv):
                if tem_copa == 0: return 0
                
                # Só há risco de soberba se o time for muito superior (ex: 10 posições de diferença)
                gap_tabela = pos_adv - pos_time
                if gap_tabela > 0:
                    # Quanto maior o gap e mais perto do fim, maior o risco de poupar
                    sinal = (gap_tabela / 20) ** 2
                    return sinal * (rodada / 38)
                return 0

            # Pegamos os dados de Copa que já calculamos antes
            c_m = 1 if listas_features['Proxima_Copa_Mandante'][-1] != '-' else 0
            c_v = 1 if listas_features['Proxima_Copa_Visitante'][-1] != '-' else 0
            
            sob_m = calcular_soberba(c_m, pos_m, pos_v)
            sob_v = calcular_soberba(c_v, pos_v, pos_m)
            
            listas_features['Soberba_Mandante'].append(sob_m)
            listas_features['Soberba_Visitante'].append(sob_v)
            listas_features['Delta_Soberba'].append(sob_m - sob_v)
            
            # --- ATUALIZAÇÃO PÓS-JOGO ---
            g_m, g_v = jogo['Gols_Mandante'], jogo['Gols_Visitante']
            res_m, res_v = ('V','D') if g_m > g_v else (('D','V') if g_v > g_m else ('E','E'))
            stats_m['ultimos_5_saldos_casa'].append(g_m - g_v); stats_v['ultimos_5_saldos_fora'].append(g_v - g_m)
            stats_m['ultimos_5_resultados'].append(res_m); stats_v['ultimos_5_resultados'].append(res_v)
            
            stats_m.update({'jogos_casa': stats_m['jogos_casa']+1, 'gols_marcados_casa': stats_m['gols_marcados_casa']+g_m, 'gols_sofridos_casa': stats_m['gols_sofridos_casa']+g_v, 'sg_casa': stats_m['sg_casa']+(g_m-g_v)})
            stats_v.update({'jogos_fora': stats_v['jogos_fora']+1, 'gols_marcados_fora': stats_v['gols_marcados_fora']+g_v, 'gols_sofridos_fora': stats_v['gols_sofridos_fora']+g_m, 'sg_fora': stats_v['sg_fora']+(g_v-g_m)})
            if g_m > g_v: stats_m['pontos'] += 3; stats_m['vitorias'] += 1
            elif g_v > g_m: stats_v['pontos'] += 3; stats_v['vitorias'] += 1
            else: stats_m['pontos'] += 1; stats_v['pontos'] += 1

    # Adiciona tudo ao DataFrame final
    # 1. Atribuição de todas as listas acumuladas para o DataFrame
    for nome, lista in listas_features.items():
        if len(lista) == len(df_jogos):
            df_jogos[nome] = lista
        else:
            # Debug caso as listas tenham tamanhos diferentes (ajuda a achar erros de loop)
            print(f"⚠️ Aviso: Coluna {nome} com tamanho {len(lista)} diferente do DF {len(df_jogos)}")

    # 2. Cálculos Derivados (Sincronização Final)
    df_jogos['Diferenca_Posicao'] = df_jogos['Posicao_Mandante'] - df_jogos['Posicao_Visitante']
    df_jogos['Equilibrio_Posicao'] = (abs(df_jogos['Diferenca_Posicao']) <= 3).astype(int)
    df_jogos['Jogo_de_6_Pontos'] = (abs(df_jogos['Diferenca_Posicao']) <= 4).astype(int)
    
    # Diferencial de Fase (Momentum)
    df_jogos['Delta_Momentum'] = df_jogos['Momentum_M'] - df_jogos['Momentum_V']
    df_jogos['Eh_Serie_B'] = listas_features['Eh_Serie_B']
    # Diferencial de Emoção (Desespero)
    # Garante que as colunas individuais existam antes de calcular o Delta
    df_jogos['Desespero_Mandante'] = listas_features['Desespero_Mandante']
    df_jogos['Desespero_Visitante'] = listas_features['Desespero_Visitante']
    df_jogos['Delta_Desespero'] = df_jogos['Desespero_Mandante'] - df_jogos['Desespero_Visitante']
    df_jogos['Soberba_Mandante'] = listas_features['Soberba_Mandante']
    df_jogos['Soberba_Visitante'] = listas_features['Soberba_Visitante']
    df_jogos['Delta_Soberba'] = df_jogos['Soberba_Mandante'] - df_jogos['Soberba_Visitante']

    # Sensores de Força Relativa
    df_jogos['Soma_Forca_Atk_Def'] = df_jogos['Forca_Atk_M'] + df_jogos['Forca_Def_V']
    df_jogos['Produto_Forca_Atk_Def'] = df_jogos['Forca_Atk_M'] * df_jogos['Forca_Def_V']
    df_jogos['Diferenca_Forca_Atk_Def'] = abs(df_jogos['Forca_Atk_M'] - df_jogos['Forca_Def_V'])
    
    # Clássico Regional
    df_jogos['É_Clássico'] = (df_jogos[COLUNA_MANDANTE].map(mapa_time_regiao) == 
                              df_jogos[COLUNA_VISITANTE].map(mapa_time_regiao)).astype(int)
    
    print(f"✅ Sucesso! DF finalizado com {len(df_jogos.columns)} colunas.")
    return df_jogos

Preparar XGBoost


In [ ]:
def preparar_para_xgboost(df):
    df_ml = df.copy()
    pastas = ['brasileiraoA', 'brasileiraoB', 'copadobrasil', 'libertadores', 'sudamericana']
    df_historico = pd.concat([pd.read_csv(f) for p in pastas for f in glob.glob(f'dados/{p}/*.csv')], ignore_index=True)


    # 1. Variável Alvo (Target)
    def definir_alvo(row):
        if row['Gols_Mandante'] > row['Gols_Visitante']: return 1
        if row['Gols_Mandante'] < row['Gols_Visitante']: return 2
        return 0
    
    df_ml['Target'] = df_ml.apply(definir_alvo, axis=1)

    # 2. Conversões de Sequência e Copa
    def converter_sequencia(seq):
        if seq == '-' or pd.isna(seq): return 0
        mapa_pts = {'V': 3, 'E': 1, 'D': 0}
        return sum(mapa_pts.get(resultado, 0) for resultado in seq)

    df_ml['Pts_Ultimos_5_M'] = df_ml['Sequencia_5_Mandante'].apply(converter_sequencia)
    df_ml['Pts_Ultimos_5_V'] = df_ml['Sequencia_5_Visitante'].apply(converter_sequencia)
    df_ml['Tem_Copa_M'] = (df_ml['Proxima_Copa_Mandante'] != '-').astype(int)
    df_ml['Tem_Copa_V'] = (df_ml['Proxima_Copa_Visitante'] != '-').astype(int)

    # 3. LISTA FINAL DE RECURSOS (30 Sensores)
    colunas_finais = [
        'Rodada', 'Posicao_Mandante', 'Posicao_Visitante', 'Diferenca_Posicao',
        'Media_GM_Casa', 'Media_GS_Casa', 'Media_GM_Fora', 'Media_GS_Fora',
        'Saldo_Gols_Casa_Mandante', 'Saldo_Gols_Fora_Visitante',
        'Saldo_Ultimos_5_Casa_Mandante', 'Saldo_Ultimos_5_Fora_Visitante',
        'Pts_Ultimos_5_M', 'Pts_Ultimos_5_V',
        'Tem_Copa_M', 'Tem_Copa_V', 'É_Clássico', 'Jogo_de_6_Pontos',
        
        # --- NOVOS SINAIS (O "SINAL LIMPO") ---
        'Forca_Atk_M', 'Forca_Def_V', 'Sinal_Dominio', 
        'Momentum_M', 'Momentum_V', 'Delta_Momentum',
        'Soma_Forca_Atk_Def', 'Produto_Forca_Atk_Def', 'Diferenca_Forca_Atk_Def', # <-- VÍRGULA CORRIGIDA AQUI
        'Desespero_Mandante', 'Desespero_Visitante', 'Delta_Desespero',
        'Soberba_Mandante', 'Soberba_Visitante', 'Delta_Soberba',
    ]

    # 4. Limpeza e Matriz Final
    X = df_ml[colunas_finais].fillna(0)
    y = df_ml['Target']

    return X, y

# --- EXECUÇÃO DO FLUXO ---
# 1. Carrega os dados
df_bruto_a = pd.read_csv(caminho_serie_a)
df_bruto_b = pd.read_csv(caminho_serie_b)

# 2. Gera as features (Sinal de 74.8% + Desespero)
df_enriquecido_a = gerar_features_completas(df_bruto_a, df_times)
df_enriquecido_b = gerar_features_completas(df_bruto_b, df_times)

# 3. Une as bases
df_total_enriquecido = pd.concat([df_enriquecido_a, df_enriquecido_b], ignore_index=True)
print(f"✅ Base unificada com {len(df_total_enriquecido)} jogos.")

# 4. Prepara para o Treino
X, y = preparar_para_xgboost(df_total_enriquecido)
print("🚀 Matriz de 30 sensores pronta para o XGBoost!")

Treino XGBoost


In [ ]:
# --- Treino ---
modelo_loteca = xgb.XGBClassifier(max_depth=6,           # Mais profundidade para pegar padrões complexos
    learning_rate=0.03, 
    n_estimators=300, 
    subsample=0.9, 
    colsample_bytree=0.9)
modelo_loteca.fit(X, y)

Acúracia

In [ ]:
from sklearn.metrics import accuracy_score

# Fazendo a previsão nos mesmos dados de treino
pred_treino = modelo_loteca.predict(X)
acuracia = accuracy_score(y, pred_treino)

print(f"📊 Acurácia no treino: {acuracia:.2%}")

Mostrar Sensores


In [ ]:
import matplotlib.pyplot as plt

# Mostra quais sensores a IA mais usou para chegar nos 14 pontos
plt.figure(figsize=(10, 8))
importancias = pd.Series(modelo_loteca.feature_importances_, index=X.columns)
importancias.nlargest(33).plot(kind='barh')
plt.title("Sensores da IA Loteca")
plt.show()

Testar IA Detalhada com 2 duplos e 1 Triplo


In [ ]:
import glob
import os
import re
import pandas as pd
import numpy as np

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def converter_sequencia(seq):
    if seq == '-' or pd.isna(seq): return 0
    mapa_pts = {'V': 3, 'E': 1, 'D': 0}
    return sum(mapa_pts.get(resultado, 0) for resultado in seq)

def executar_simulacao_ia_smart_order(diretorio, df_gabarito, modelo_ia, threshold):
    arquivos = sorted(glob.glob(os.path.join(diretorio, 'rodada*.csv')), key=natural_sort_key)
    mapa_res = {1: "CASA  ", 0: "EMPATE", 2: "FORA  "}

    for caminho in arquivos:
        df_concurso = pd.read_csv(caminho)
        nome_arquivo = os.path.basename(caminho)
        jogos_analise = []

        # 1. COLETA DE DADOS (Sem ordenar ainda)
        for i, jogo in df_concurso.iterrows():
            m, v = jogo['Time da Casa'], jogo['Time Visitante']
            jr = df_gabarito[(df_gabarito['Time da Casa'] == m) & (df_gabarito['Time Visitante'] == v)]
            if jr.empty: continue
            
            real_num = 1 if jr['Gols_Mandante'].values[0] > jr['Gols_Visitante'].values[0] else (2 if jr['Gols_Visitante'].values[0] > jr['Gols_Mandante'].values[0] else 0)
            stats_m = df_gabarito[(df_gabarito['Time da Casa'] == m) | (df_gabarito['Time Visitante'] == m)].tail(1)
            stats_v = df_gabarito[(df_gabarito['Time da Casa'] == v) | (df_gabarito['Time Visitante'] == v)].tail(1)

            # (O input_ia deve conter todas as 24 features conforme o teu treino anterior)
            input_ia = pd.DataFrame([{
                'Rodada': jogo['Rodada'], 'Posicao_Mandante': stats_m['Posicao_Mandante'].values[0],
                'Posicao_Visitante': stats_v['Posicao_Visitante'].values[0],
                'Diferenca_Posicao': stats_m['Posicao_Mandante'].values[0] - stats_v['Posicao_Visitante'].values[0],
                'Media_GM_Casa': stats_m['Media_GM_Casa'].values[0], 'Media_GS_Casa': stats_m['Media_GS_Casa'].values[0],
                'Media_GM_Fora': stats_v['Media_GM_Fora'].values[0], 'Media_GS_Fora': stats_v['Media_GS_Fora'].values[0],
                'Saldo_Gols_Casa_Mandante': stats_m['Saldo_Gols_Casa_Mandante'].values[0],
                'Saldo_Gols_Fora_Visitante': stats_v['Saldo_Gols_Fora_Visitante'].values[0],
                'Saldo_Ultimos_5_Casa_Mandante': stats_m['Saldo_Ultimos_5_Casa_Mandante'].values[0],
                'Saldo_Ultimos_5_Fora_Visitante': stats_v['Saldo_Ultimos_5_Fora_Visitante'].values[0],
                'Pts_Ultimos_5_M': converter_sequencia(stats_m['Sequencia_5_Mandante'].values[0]),
                'Pts_Ultimos_5_V': converter_sequencia(stats_v['Sequencia_5_Visitante'].values[0]),
                'Tem_Copa_M': 1 if stats_m['Proxima_Copa_Mandante'].values[0] != '-' else 0,
                'Tem_Copa_V': 1 if stats_v['Proxima_Copa_Visitante'].values[0] != '-' else 0,
                'É_Clássico': jogo['É_Clássico'] if 'É_Clássico' in jogo else 0,
                'Jogo_de_6_Pontos': 1 if abs(stats_m['Posicao_Mandante'].values[0] - stats_v['Posicao_Visitante'].values[0]) <= 4 else 0,
                'Forca_Atk_M': stats_m['Forca_Atk_M'].values[0], 'Forca_Def_V': stats_v['Forca_Def_V'].values[0],
                'Sinal_Dominio': stats_m['Sinal_Dominio'].values[0], 'Momentum_M': stats_m['Momentum_M'].values[0],
                'Momentum_V': stats_v['Momentum_V'].values[0], 'Delta_Momentum': stats_m['Momentum_M'].values[0] - stats_v['Momentum_V'].values[0],
                'Soma_Forca_Atk_Def': stats_m['Soma_Forca_Atk_Def'].values[0],
                'Produto_Forca_Atk_Def': stats_m['Produto_Forca_Atk_Def'].values[0],
                'Diferenca_Forca_Atk_Def': abs(stats_m['Forca_Atk_M'].values[0] - stats_v['Forca_Def_V'].values[0]),
                'Desespero_Mandante': stats_m['Desespero_Mandante'].values[0],
                'Desespero_Visitante': stats_v['Desespero_Visitante'].values[0],
                'Delta_Desespero': stats_m['Delta_Desespero'].values[0],
                'Soberba_Mandante': stats_m['Soberba_Mandante'].values[0],
                'Soberba_Visitante': stats_v['Soberba_Visitante'].values[0],
                'Delta_Soberba': stats_m['Delta_Soberba'].values[0]
            }])

            probs = modelo_ia.predict_proba(input_ia)[0]
            ordem = np.argsort(probs)[::-1]
            p_primario = 0 if abs(probs[1] - probs[2]) < threshold else np.argmax(probs)
            p_secundario = ordem[1] if ordem[0] == p_primario else ordem[0]

            jogos_analise.append({
                'id_original': i, # Mantém a ordem do bilhete
                'confronto': f"{m[:12]} x {v[:12]}", 'real': real_num, 
                'p1': p_primario, 'p2': p_secundario,
                'gap': abs(probs[1] - probs[2]), 'probs': probs,
                'p_casa': probs[1], 'p_empate': probs[0], 'p_fora': probs[2],
                'tipo': 'S' # Começa tudo como Seco
            })

        # 2. ALOCAÇÃO INTELIGENTE DE RECURSOS
        # Ordenamos temporariamente para decidir onde gastar
        jogos_analise.sort(key=lambda x: x['gap'])
        
        for idx, j in enumerate(jogos_analise):
            if idx == 0: j['tipo'] = 'T' # O mais incerto de todos leva o TRIPLO
            elif idx < 3: j['tipo'] = 'D' # Os próximos 6 levam DUPLO

        # 3. VOLTA À ORDEM ORIGINAL E IMPRIME
        jogos_analise.sort(key=lambda x: x['id_original'])

        print(f"\n# {nome_arquivo.upper()} - BILHETE FINAL #")
        print(f"{'J':<2} | {'CONFRONTO':<32} | {'TIPO':<4} | {'CASA%':<6} | {'EMP%':<6} | {'FORA%':<6} | {'PALPITE':<18} | {'REAL':<8} | {'STATUS'}")        
        print("-" * 130)

        acertos = 0
        for idx, j in enumerate(jogos_analise):
            p_max = np.max(j['probs'])
            
            # Validação
            acertou = (j['tipo'] == "T") or (j['tipo'] == "D" and (j['p1'] == j['real'] or j['p2'] == j['real'])) or (j['tipo'] == "S" and j['p1'] == j['real'])
            if acertou: acertos += 1
            
            palpite_str = "TRIPLO" if j['tipo'] == "T" else (f"{mapa_res[j['p1']]} / {mapa_res[j['p2']]}" if j['tipo'] == "D" else mapa_res[j['p1']])
            status = "✅" if acertou else "❌"
            print(f"{idx+1:<2} | {j['confronto']:<32} | {j['tipo']:<4} | {j['p_casa']:>6.1%} | {j['p_empate']:>6.1%} | {j['p_fora']:>6.1%} | {palpite_str:<18} |{mapa_res[j['real']]:<8}| {status}")
            #print(f"{idx+1:<2} | {j['confronto']:<28} | {j['tipo']:<4} | {p_max:.1%} | {palpite_str:<18} | {mapa_res[j['real']]:<8} | {status}")

        print(f"\n📊 TOTAL: {acertos}/14")
        print(f"📈 ACURÁCIA: {acertos/14:.1%}")


executar_simulacao_ia_smart_order('simulacao/2006', df_total_enriquecido, modelo_loteca, threshold=0.01)

In [ ]:


# Gerar features para o Treino
train_15_a = gerar_features_completas(df_15_a, df_times_15)
train_15_b = gerar_features_completas(df_15_b, df_times_15)
train_16_a = gerar_features_completas(df_16_a, df_times_16)
train_16_b = gerar_features_completas(df_16_b, df_times_16)

# Base de Treino Unificada
df_treino_total = pd.concat([train_15_a, train_15_b, train_16_a, train_16_b], ignore_index=True)
X_train, y_train = preparar_para_xgboost(df_treino_total)

# --- CARREGAMENTO PARA GABARITO (2006) ---
df_06_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2006.csv')
df_06_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2006.csv')

# Gerar features para o Gabarito (Essencial para o simulador encontrar os jogos)
df_gabarito_06 = pd.concat([
    gerar_features_completas(df_06_a, df_times_06),
    gerar_features_completas(df_06_b, df_times_06)
], ignore_index=True)

# --- TREINAR O MODELO ---
modelo_loteca = xgb.XGBClassifier(
    max_depth=8,           # Menos profundidade = mais generalização
    learning_rate=0.02, 
    n_estimators=400,
    subsample=0.8,         # Treina com pedaços aleatórios para não viciar
    colsample_bytree=0.8,
    gamma=1)
modelo_loteca.fit(X_train, y_train)

print("✅ Modelo treinado com 2015/2016 e pronto para testar 2006!")

# Chamada corrigida
executar_simulacao_ia_smart_order(
    diretorio='simulacao/2006', 
    df_gabarito=df_gabarito_06, 
    modelo_ia=modelo_loteca, 
    threshold=0.01
)

In [ ]:
# --- 1. FUNÇÃO DE PREPARAÇÃO CORRIGIDA (Calcula os Deltas e Sinais Derivados) ---
 # --- CARREGAMENTO PARA TREINO (2015 e 2016) ---

# Carregue os CSVs de 2015 e 2016 (A e B)

# --- CARREGAMENTO COM MARCAÇÃO DE SÉRIE ---

df_15_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2015.csv')
df_15_a['Eh_Serie_B'] = 0 # Série A

df_15_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2015.csv')
df_15_b['Eh_Serie_B'] = 1 # Série B

df_16_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2016.csv')
df_16_a['Eh_Serie_B'] = 0
df_16_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2016.csv')
df_16_b['Eh_Serie_B'] = 1

# Carregue os times de cada ano (importante para as regiões)
df_times_06 = pd.read_csv('dados/times/times2020.csv')
df_times_15 = pd.read_csv('dados/times/times2015.csv') # Você precisará desse arquivo
df_times_16 = pd.read_csv('dados/times/times2016.csv') # E desse

# Gerar features para o Treino
train_15_a = gerar_features_completas(df_15_a, df_times_15)
train_15_b = gerar_features_completas(df_15_b, df_times_15)
train_16_a = gerar_features_completas(df_16_a, df_times_16)
train_16_b = gerar_features_completas(df_16_b, df_times_16)

# Base de Treino Unificada
df_treino_total = pd.concat([train_15_a, train_15_b, train_16_a, train_16_b], ignore_index=True)
X_train, y_train = preparar_para_xgboost(df_treino_total)

# --- CARREGAMENTO PARA GABARITO (2006) ---
df_06_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2020.csv')
df_06_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2020.csv')
# Gerar features para o Gabarito (Essencial para o simulador encontrar os jogos)

def normalizar_times1(nome):
    nome = str(nome).strip()
    substituicoes = {
        'Internaciona': 'Internacional', 'Sampaio Corr': 'Sampaio Corrêa',
        'Atletico-GO': 'Atlético-GO', 'Atlético-MG': 'Atlético-MG',
        'Athletico-PR': 'Atlético-PR', 'Athletico Paranaense': 'Atlético-PR',
        'Ceará SC': 'Ceará', 'Sport Recife': 'Sport', 'Vasco da Gama': 'Vasco',
        'América Mineiro': 'América-MG', 'Red Bull Bragantino': 'Bragantino',
        'Grêmio Novorizontino': 'Novorizontino'
    }
    for curto, completo in substituicoes.items():
        if nome.startswith(curto): return completo
    return nome

df_gabarito_06 = pd.concat([
    gerar_features_completas(df_06_a, df_times_06),
    gerar_features_completas(df_06_b, df_times_06)
], ignore_index=True) 

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

def converter_sequencia(seq):
    if seq == '-' or pd.isna(seq): return 0
    mapa_pts = {'V': 3, 'E': 1, 'D': 0}
    return sum(mapa_pts.get(resultado, 0) for resultado in seq)

def preparar_dados_ml_calibrado(df):
    df = df.copy()
    
    # Target: 0=Empate, 1=Casa, 2=Fora
    # Verificamos se as colunas existem antes de calcular
    col_gm = 'Gols_Mandante' if 'Gols_Mandante' in df.columns else 'Gols_M'
    col_gv = 'Gols_Visitante' if 'Gols_Visitante' in df.columns else 'Gols_V'
    
    df['Target'] = df.apply(lambda r: 1 if r[col_gm] > r[col_gv] else (2 if r[col_gv] > r[col_gm] else 0), axis=1)
    
    # Pesos para equilibrar vitórias fora
    df['Pesos'] = df['Target'].apply(lambda x: 2.0 if x == 2 else 1.0)

    # --- Sincronização de Sensores (Garantindo nomes consistentes) ---
    df['Tem_Copa_M'] = df['Proxima_Copa_Mandante'] if 'Proxima_Copa_Mandante' in df.columns else 0
    df['Tem_Copa_V'] = df['Proxima_Copa_Visitante'] if 'Proxima_Copa_Visitante' in df.columns else 0
    df['Diferenca_Posicao'] = df['Posicao_Mandante'] - df['Posicao_Visitante']
    df['Delta_Momentum'] = df['Momentum_M'] - df['Momentum_V']
    df['Soma_Forca_Atk_Def'] = df['Forca_Atk_M'] + df['Forca_Def_V']
    df['Produto_Forca_Atk_Def'] = df['Forca_Atk_M'] * df['Forca_Def_V']
    df['Diferenca_Forca_Atk_Def'] = abs(df['Forca_Atk_M'] - df['Forca_Def_V'])
    df['Jogo_de_6_Pontos'] = (abs(df['Diferenca_Posicao']) <= 4).astype(int)
    
    def conv_seq(s):
        if s == '-' or pd.isna(s): return 0
        return sum({'V': 3, 'E': 1, 'D': 0}.get(res, 0) for res in str(s))
    
    df['Pts_Ultimos_5_M'] = df['Sequencia_5_Mandante'].apply(conv_seq)
    df['Pts_Ultimos_5_V'] = df['Sequencia_5_Visitante'].apply(conv_seq)

    # LISTA DE SENSORES
    cols = [
        'Rodada', 'Posicao_Mandante', 'Posicao_Visitante', 'Diferenca_Posicao',
        'Media_GM_Casa', 'Media_GS_Casa', 'Media_GM_Fora', 'Media_GS_Fora',
        'Saldo_Gols_Casa_Mandante', 'Saldo_Gols_Fora_Visitante',
        'Saldo_Ultimos_5_Casa_Mandante', 'Saldo_Ultimos_5_Fora_Visitante',
        'Pts_Ultimos_5_M', 'Pts_Ultimos_5_V',
        'Tem_Copa_M', 'Tem_Copa_V', 'É_Clássico', 'Jogo_de_6_Pontos',
        'Forca_Atk_M', 'Forca_Def_V', 'Sinal_Dominio', 
        'Momentum_M', 'Momentum_V', 'Delta_Momentum',
        'Soma_Forca_Atk_Def', 'Produto_Forca_Atk_Def', 'Diferenca_Forca_Atk_Def',
        'Desespero_Mandante', 'Desespero_Visitante', 'Delta_Desespero',
        'Soberba_Mandante', 'Soberba_Visitante', 'Delta_Soberba', 'Eh_Serie_B'
    ]
    
    # --- 🚀 A SOLUÇÃO DO ERRO AQUI: ---
    # Forçamos todas as colunas de sensores a serem numéricas. 
    # Qualquer string perdida vira NaN e depois vira 0 pelo fillna.
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df[cols].fillna(0), df['Target'], df['Pesos'], cols

# --- 2. EXECUÇÃO DO TREINO ---
# Agora usamos a função calibrada para gerar as colunas finais
X, y, pesos, colunas_finais = preparar_dados_ml_calibrado(df_treino_total)

modelo_calibrado = xgb.XGBClassifier(
    max_depth=4, 
    learning_rate=0.02, 
    n_estimators=400,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    eval_metric='mlogloss'
).fit(X, y, sample_weight=pesos)

# --- 3. SIMULAÇÃO ATUALIZADA (Garante que o gabarito também tenha os novos sensores) ---
def executar_simulacao_agressiva(diretorio, df_gabarito, modelo_ia, colunas):
    # Pré-processa o gabarito com a mesma lógica de sensores derivados do treino
    # Isso evita que a IA tente ler colunas que não existem no gabarito
    df_gab_analise, _, _, _ = preparar_dados_ml_calibrado(df_gabarito)
    # Re-adicionamos os nomes para o lookup funcionar
    df_gab_analise['Time da Casa'] = df_gabarito['Time da Casa'].values
    df_gab_analise['Time Visitante'] = df_gabarito['Time Visitante'].values
    df_gab_analise['G_M_Real'] = df_gabarito['Gols_Mandante'].values
    df_gab_analise['G_V_Real'] = df_gabarito['Gols_Visitante'].values

    arquivos = sorted(glob.glob(os.path.join(diretorio, 'rodada*.csv')), key=natural_sort_key)
    mapa_res = {1: "CASA  ", 0: "EMPATE", 2: "FORA  "}

    for caminho in arquivos:
        df_c = pd.read_csv(caminho)
        df_c['Time da Casa'] = df_c['Time da Casa'].apply(normalizar_times)
        df_c['Time Visitante'] = df_c['Time Visitante'].apply(normalizar_times)
        jogos_analise = []

        for i, jogo in df_c.iterrows():
            m, v = jogo['Time da Casa'], jogo['Time Visitante']
            jr = df_gab_analise[(df_gab_analise['Time da Casa'] == m) & (df_gab_analise['Time Visitante'] == v)]
            
            if jr.empty: continue
            
            # Pega a linha exata com os 34 sensores
            input_ia = jr[colunas].tail(1)
            probs = modelo_ia.predict_proba(input_ia)[0]
            
            # Filtro Michel (Equilíbrio 40/40)
            p_casa, p_emp, p_fora = probs[1], probs[0], probs[2]
            if abs(p_casa - p_fora) < 0.03: # Threshold de 3%
                p1, p2 = 0, (1 if p_casa >= p_fora else 2)
            else:
                ordem = np.argsort(probs)[::-1]
                p1, p2 = ordem[0], ordem[1]

            real = 1 if jr['G_M_Real'].values[0] > jr['G_V_Real'].values[0] else \
                   (2 if jr['G_V_Real'].values[0] > jr['G_M_Real'].values[0] else 0)

            jogos_analise.append({
                'id_original': i, # Mantém a ordem do bilhete
                'confronto': f"{m[:12]} x {v[:12]}", 'real': real, 
                'p1': p1, 'p2': p2,
                'gap': abs(probs[1] - probs[2]), 'probs': probs,
                'p_casa': probs[1], 'p_empate': probs[0], 'p_fora': probs[2],
                'tipo': 'S' # Começa tudo como Seco
            })
            
        jogos_analise.sort(key=lambda x: x['gap'])
        
        for idx, j in enumerate(jogos_analise):
            if idx == 0: j['tipo'] = 'T' # O mais incerto de todos leva o TRIPLO
            elif idx < 3: j['tipo'] = 'D' # Os próximos 6 levam DUPLO
            
        jogos_analise.sort(key=lambda x: x['id_original'])

        print(f"{'J':<2} | {'CONFRONTO':<32} | {'TIPO':<4} | {'CASA%':<6} | {'EMP%':<6} | {'FORA%':<6} | {'PALPITE':<18} | {'REAL':<8} | {'STATUS'}")        
        print("-" * 130)

        acertos = 0
        for idx, j in enumerate(jogos_analise):
            p_max = np.max(j['probs'])
            
            # Validação
            acertou = (j['tipo'] == "T") or (j['tipo'] == "D" and (j['p1'] == j['real'] or j['p2'] == j['real'])) or (j['tipo'] == "S" and j['p1'] == j['real'])
            if acertou: acertos += 1
            
            palpite_str = "TRIPLO" if j['tipo'] == "T" else (f"{mapa_res[j['p1']]} / {mapa_res[j['p2']]}" if j['tipo'] == "D" else mapa_res[j['p1']])
            status = "✅" if acertou else "❌"
            print(f"{idx+1:<2} | {j['confronto']:<32} | {j['tipo']:<4} | {j['p_casa']:>6.1%} | {j['p_empate']:>6.1%} | {j['p_fora']:>6.1%} | {palpite_str:<18} |{mapa_res[j['real']]:<8}| {status}")
            #print(f"{idx+1:<2} | {j['confronto']:<28} | {j['tipo']:<4} | {p_max:.1%} | {palpite_str:<18} | {mapa_res[j['real']]:<8} | {status}")

        print(f"\n📊 TOTAL: {acertos}/14")
        print(f"📈 ACURÁCIA: {acertos/14:.1%}")

# Chamada final
executar_simulacao_agressiva('simulacao/2020', df_gabarito_06, modelo_calibrado, colunas_finais)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import glob
import os
import re
from collections import deque

# --- 1. AUXILIARES E NORMALIZAÇÃO ---
def normalizar_times(nome):
    nome = str(nome).strip()
    substituicoes = {
        'Internaciona': 'Internacional', 'Sampaio Corr': 'Sampaio Corrêa',
        'Atletico-GO': 'Atlético-GO', 'Atlético-MG': 'Atlético-MG',
        'Athletico-PR': 'Atlético-PR', 'Athletico Paranaense': 'Atlético-PR',
        'Ceará SC': 'Ceará', 'Sport Recife': 'Sport', 'Vasco da Gama': 'Vasco',
        'América Mineiro': 'América-MG', 'Red Bull Bragantino': 'Bragantino',
        'Grêmio Novorizontino': 'Novorizontino', 'Athletico Paranaens': 'Atlético-PR'
    }
    for curto, completo in substituicoes.items():
        if nome.startswith(curto): return completo
    return nome

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('([0-9]+)', s)]

# --- 2. PREPARAÇÃO DOS 34 SENSORES (Calcula tudo o que o modelo precisa) ---
def preparar_dados_ml_calibrado(df):
    df = df.copy()
    
    # Identifica colunas de gols e cria o Alvo
    c_m = 'Gols_Mandante' if 'Gols_Mandante' in df.columns else 'Gols_M'
    c_v = 'Gols_Visitante' if 'Gols_Visitante' in df.columns else 'Gols_V'
    df['Target'] = df.apply(lambda r: 1 if r[c_m] > r[c_v] else (2 if r[c_v] > r[c_m] else 0), axis=1)
    df['Pesos'] = df['Target'].apply(lambda x: 2.0 if x == 2 else 1.0)

    # Cálculo de Sensores Derivados (Os 34 Sensores)
    df['Diferenca_Posicao'] = df['Posicao_Mandante'] - df['Posicao_Visitante']
    df['Delta_Momentum'] = df['Momentum_M'] - df['Momentum_V']
    df['Soma_Forca_Atk_Def'] = df['Forca_Atk_M'] + df['Forca_Def_V']
    df['Produto_Forca_Atk_Def'] = df['Forca_Atk_M'] * df['Forca_Def_V']
    df['Diferenca_Forca_Atk_Def'] = abs(df['Forca_Atk_M'] - df['Forca_Def_V'])
    df['Jogo_de_6_Pontos'] = (abs(df['Diferenca_Posicao']) <= 4).astype(int)
    df['Tem_Copa_M'] = df['Proxima_Copa_Mandante'] if 'Proxima_Copa_Mandante' in df.columns else 0
    df['Tem_Copa_V'] = df['Proxima_Copa_Visitante'] if 'Proxima_Copa_Visitante' in df.columns else 0
    df['Delta_Desespero'] = df['Desespero_Mandante'] - df['Desespero_Visitante']
    
    # Se existirem colunas de Soberba, calcula o delta
    if 'Soberba_Mandante' in df.columns:
        df['Delta_Soberba'] = df['Soberba_Mandante'] - df['Soberba_Visitante']
    else:
        df['Delta_Soberba'] = 0; df['Soberba_Mandante'] = 0; df['Soberba_Visitante'] = 0

    # Conversão de Sequência para Pontuação
    def conv_seq(s):
        if s == '-' or pd.isna(s): return 0
        return sum({'V': 3, 'E': 1, 'D': 0}.get(res, 0) for res in str(s))
    
    df['Pts_Ultimos_5_M'] = df['Sequencia_5_Mandante'].apply(conv_seq)
    df['Pts_Ultimos_5_V'] = df['Sequencia_5_Visitante'].apply(conv_seq)

    cols = [
        'Rodada', 'Posicao_Mandante', 'Posicao_Visitante', 'Diferenca_Posicao',
        'Media_GM_Casa', 'Media_GS_Casa', 'Media_GM_Fora', 'Media_GS_Fora',
        'Saldo_Gols_Casa_Mandante', 'Saldo_Gols_Fora_Visitante',
        'Saldo_Ultimos_5_Casa_Mandante', 'Saldo_Ultimos_5_Fora_Visitante',
        'Pts_Ultimos_5_M', 'Pts_Ultimos_5_V', 'Tem_Copa_M', 'Tem_Copa_V', 
        'É_Clássico', 'Jogo_de_6_Pontos', 'Forca_Atk_M', 'Forca_Def_V', 'Sinal_Dominio', 
        'Momentum_M', 'Momentum_V', 'Delta_Momentum', 'Soma_Forca_Atk_Def', 
        'Produto_Forca_Atk_Def', 'Diferenca_Forca_Atk_Def', 'Desespero_Mandante', 
        'Desespero_Visitante', 'Delta_Desespero', 'Soberba_Mandante', 
        'Soberba_Visitante', 'Delta_Soberba', 'Eh_Serie_B'
    ]
    
    # Força tudo para numérico (evita erro de 'object' no XGBoost)
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    return df[cols].fillna(0), df['Target'], df['Pesos'], cols

# --- 3. EXECUÇÃO DO CARREGAMENTO E TREINO ---
print("⚙️  Carregando e processando dados de treino...")
df_15_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2015.csv'); df_15_a['Eh_Serie_B'] = 0
df_15_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2015.csv'); df_15_b['Eh_Serie_B'] = 1
df_16_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2016.csv'); df_16_a['Eh_Serie_B'] = 0
df_16_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2016.csv'); df_16_b['Eh_Serie_B'] = 1

t15 = pd.read_csv('dados/times/times2015.csv'); t16 = pd.read_csv('dados/times/times2016.csv')

# União da base de treino
train_total = pd.concat([
    gerar_features_completas(df_15_a, t15), gerar_features_completas(df_15_b, t15),
    gerar_features_completas(df_16_a, t16), gerar_features_completas(df_16_b, t16)
], ignore_index=True)

X_train, y_train, pesos, colunas_finais = preparar_dados_ml_calibrado(train_total)

print("🚀 Treinando Comitê de Modelos...")
m_xgb = xgb.XGBClassifier(max_depth=4, learning_rate=0.02, n_estimators=400, subsample=0.8, colsample_bytree=0.8, objective='multi:softprob').fit(X_train, y_train, sample_weight=pesos)
m_rf = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_leaf=5).fit(X_train, y_train)

# --- 4. PREPARAÇÃO DO GABARITO 2020 (Super Gabarito A + B) ---
print("🎯 Preparando Super Gabarito 2020...")
df_20_a = pd.read_csv('dados/brasileiraoA/brasileiraoA2010.csv'); df_20_a['Eh_Serie_B'] = 0
df_20_b = pd.read_csv('dados/brasileiraoB/brasileiraoB2010.csv'); df_20_b['Eh_Serie_B'] = 1
t20 = pd.read_csv('dados/times/times2010.csv')

# Unifica Gabarito
df_gab_raw = pd.concat([gerar_features_completas(df_20_a, t20), gerar_features_completas(df_20_b, t20)], ignore_index=True)
# Aplica os sensores derivados ao gabarito
X_gab, _, _, _ = preparar_dados_ml_calibrado(df_gab_raw)
# Re-atribui os nomes para o simulador conseguir buscar
X_gab['Time da Casa'] = df_gab_raw['Time da Casa'].values
X_gab['Time Visitante'] = df_gab_raw['Time Visitante'].values
X_gab['G_M_Real'] = df_gab_raw['Gols_Mandante'].values
X_gab['G_V_Real'] = df_gab_raw['Gols_Visitante'].values

# --- 5. SIMULAÇÃO COM COMITÊ E FILTRO DE 14 JOGOS ---
def executar_simulacao_comite(diretorio, df_gab_analise, modelo_xgb, modelo_rf, colunas, threshold=0.04):
    arquivos = sorted(glob.glob(os.path.join(diretorio, 'rodada*.csv')), key=natural_sort_key)
    mapa_res = {1: "CASA  ", 0: "EMPATE", 2: "FORA  "}

    for caminho in arquivos:
        df_c = pd.read_csv(caminho)
        nome_arquivo = os.path.basename(caminho)
        df_c['Time da Casa'] = df_c['Time da Casa'].apply(normalizar_times)
        df_c['Time Visitante'] = df_c['Time Visitante'].apply(normalizar_times)
        
        analise = []
        for i, jogo in df_c.iterrows():
            m, v = jogo['Time da Casa'], jogo['Time Visitante']
            # Busca flexível no Super Gabarito
            jr = df_gab_analise[(df_gab_analise['Time da Casa'] == m) & (df_gab_analise['Time Visitante'] == v)]
            
            if jr.empty:
                print(f"⚠️ Jogo não encontrado: {m} x {v}")
                continue
            
            input_ia = jr[colunas].tail(1)
            
            # VOTAÇÃO DO COMITÊ (Média XGB e RF)
            p_xgb = modelo_xgb.predict_proba(input_ia)[0]
            p_rf = modelo_rf.predict_proba(input_ia)[0]
            probs = (p_xgb * 0.5) + (p_rf * 0.5)
            
            p_casa, p_emp, p_fora = probs[1], probs[0], probs[2]

            
            if p_emp > 0.3:
                p1 = 0  # Força o palpite principal no Empate
                p2 = 1 if p_casa >= p_fora else 2 # Segundo palpite no "menos pior"
            else:
                ordem = np.argsort(probs)[::-1]
                p1, p2 = ordem[0], ordem[1]

            real = 1 if jr['G_M_Real'].values[0] > jr['G_V_Real'].values[0] else \
                   (2 if jr['G_V_Real'].values[0] > jr['G_M_Real'].values[0] else 0)

            analise.append({
                'id_original': i, 'confronto': f"{m[:14]} x {v[:14]}", 'real': real, 
                'p1': p1, 'p2': p2, 'gap': abs(p_casa - p_fora), 'probs': probs, 'tipo': 'S'
            })
        # Alocação de Triplo e Duplos baseada no GAP de incerteza
        analise.sort(key=lambda x: x['gap'])
        for idx, j in enumerate(analise):
            if idx == 0: j['tipo'] = 'T'
            elif idx < 4: j['tipo'] = 'D'

        analise.sort(key=lambda x: x['id_original'])
        print(f"\n# {nome_arquivo.upper()} - COMITÊ #")
        print(f"{'J':<2} | {'CONFRONTO':<32} | {'TIPO':<4} | {'CASA%':<6} | {'EMP%':<6} | {'FORA%':<6} | {'PALPITE':<18} | {'REAL':<8} | {'ST'}")        
        print("-" * 135)

        acertos = 0
        for idx, j in enumerate(analise):
            acertou = (j['tipo'] == "T") or (j['tipo'] == "D" and (j['p1'] == j['real'] or j['p2'] == j['real'])) or (j['tipo'] == "S" and j['p1'] == j['real'])
            if acertou: acertos += 1
            palpite = "TRIPLO" if j['tipo'] == "T" else (f"{mapa_res[j['p1']]} / {mapa_res[j['p2']]}" if j['tipo'] == "D" else mapa_res[j['p1']])
            print(f"{j['id_original']+1:<2} | {j['confronto']:<32} | {j['tipo']:<4} | {j['probs'][1]:>6.1%} | {j['probs'][0]:>6.1%} | {j['probs'][2]:>6.1%} | {palpite:<18} | {mapa_res[j['real']]:<8} | {'✅' if acertou else '❌'}")

        print(f"📊 RESULTADO FINAL: {acertos}/14")

# Chamada do Simulador
executar_simulacao_comite('simulacao/2010', X_gab, m_xgb, m_rf, colunas_finais)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def plotar_importancia_comite_vanilla(modelo_xgb, modelo_rf, colunas):
    # 1. Extrair e calcular média
    imp_xgb = modelo_xgb.feature_importances_
    imp_rf = modelo_rf.feature_importances_
    df_imp = pd.DataFrame({
        'Feature': colunas,
        'Média': (imp_xgb + imp_rf) / 2
    }).sort_values(by='Média', ascending=True) # Ascending True para as maiores ficarem no topo do gráfico horizontal

    # 2. Plot básico com Matplotlib
    plt.figure(figsize=(10, 12))
    plt.barh(df_imp['Feature'], df_imp['Média'], color='skyblue', edgecolor='navy')
    
    plt.title('Importância Média dos Sensores (XGB + RF)', fontsize=14)
    plt.xlabel('Ganho de Informação')
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Adicionar os nomes das features e valores
    for i, v in enumerate(df_imp['Média']):
        plt.text(v, i, f' {v:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.show()

# Chamada
plotar_importancia_comite_vanilla(m_xgb, m_rf, colunas_finais)